### Assignment 4, Tic-Tac-Toe

Your homework must be implemented in this Notebook file. 
You can add as many cells as you want. However, you are not allowed to touch the code below the line "=============".


In [1]:
###
#GENERAL HELPER FUNCTIONS
###

#Returns a list of the next possible actions a player can make based on the specified state
def generateNextPossibleStatesForStateAndPlayer(state,playerChar):
    emptySquares = indicesOfSquaresInStateWithLetter(state,'*')
    nextStates = []
    for square in emptySquares:
        newState = list(state)
        newState[square] = playerChar #place char
        nextStates.append({"placement":Placement(square,playerChar),"state":tuple(newState)})
    return nextStates

#Returns a set of the indices of squares containing the specified letter in the given state
def indicesOfSquaresInStateWithLetter(state,letter):
    indices = []
    currentIndex = 0
    for tile in state:
        if tile == letter:
            indices.append(currentIndex)
        currentIndex += 1
    return indices

#Win checking logic; checks if the current state contains 3 of the player's characters in a row
def isWinStateForPlayer(state,playerChar):
    winStates = set([(0,1,2),(3,4,5),(6,7,8),(0,3,6),(1,4,7),(2,5,8),(0,4,8),(2,4,6)])
    squaresWithPlayerChar = set(indicesOfSquaresInStateWithLetter(state,playerChar))
    for state in winStates:
        winningSetOfIndices = set(state)
        if squaresWithPlayerChar.intersection(winningSetOfIndices) == winningSetOfIndices:
            return True
    return False

#Checks whether there are more available moves to be made from the state
#Returns False if there are more moves, True if there are no more moves
def noMoreMovesInState(state):
    for square in state:
        if square == '*':
            return False
    return True

#Evaluates whether the state is a terminal state or not
#If it is a terminal state, returns the utility value as determined by the player
#If it is not a terminal state, returns None
def evaluateTerminalConditionsForStateAndPlayer(state, playerChar):
    opposingPlayerChar = opposingPlayerTo(playerChar)
    #check terminal state conditions
    if isWinStateForPlayer(state,playerChar): #win
        return 10
    elif isWinStateForPlayer(state,opposingPlayerChar): #lose
        return -10
    elif noMoreMovesInState(state): #draw
        return 0
    return None #return None if not a terminal state

#Determine opposing player
def opposingPlayerTo(playerChar):
    return 'O' if (playerChar == 'X') else 'X'

In [2]:
###
#DATA STRUCTURES
###

#node structure used for maintaining the explored portion of the transition model
class Node:
    def __init__(self, parent, state, action):
        self.parent = parent #parent Node
        self.state = state #state, as a tuple
        self.action = action #action, as a Placement
        
        
class MinMaxNode(Node):
    def __init__(self, parent, state, action, utility, depth):
        Node.__init__(self, parent, state, action)
        self.utility = utility #utility, as an integer
        self.depth = depth #depth, as an integer
        
    #override comparison based on utility
    def __lt__(self,other):
        if isinstance(other,MinMaxNode):
            return self.utility < other.utility #allows for comparison to other MinMaxNodes
        return self.utility < other #allows for comparison to numeric types
    
    #overriding comparison based on utility
    def __gt__(self,other):
        if isinstance(other,MinMaxNode):
            return self.utility > other.utility #allows for comparison to other MinMaxNodes
        return self.utility > other #allows for comparison to numeric types
    
#just a nice data structure to hold and pass around a position and playerChar
class Placement:
    def __init__(self, position, playerChar):
        self.position = position
        self.playerChar = playerChar
    
class Board:
    #initial the board to be empty
    def __init__(self):
        self.board = ['*','*','*','*','*','*','*','*','*']
        
    #takes a Placement and attempts to make a move based on that
    #breaks if the position isn't empty, but does not provide logic to recover from it
    def makeMove(self,placement):
        if self.board[placement.position] != '*':
            print('you can\'t make that move you silly goose')
            return
        self.board[placement.position] = placement.playerChar
        return
        
    #prints the current board nicely
    def printBoard(self):
        print('{0}{1}{2}'.format(self.board[0],self.board[1],self.board[2]))
        print('{0}{1}{2}'.format(self.board[3],self.board[4],self.board[5]))
        print('{0}{1}{2}'.format(self.board[6],self.board[7],self.board[8]))
        return

In [3]:
#return a placement
def minimaxDecision(state,playerChar):
    #figure out the next possible states for the current player
    nextPossibleStatesForPlayer = generateNextPossibleStatesForStateAndPlayer(state,playerChar)
    nextPossibleActionsForPlayer = []
    #add all of the states to a list
    for state in nextPossibleStatesForPlayer:
        nextPossibleActionsForPlayer.append(MinMaxNode(None,state["state"],state["placement"],None,0))
    if playerChar == 'X': # X turn - start with a max node 
        for node in nextPossibleActionsForPlayer:
            node.utility = minValue(node)
        return max(nextPossibleActionsForPlayer).action
    else: # O turn - start with a min node
        for node in nextPossibleActionsForPlayer:
            node.utility = maxValue(node)
        return min(nextPossibleActionsForPlayer).action

#takes a node and returns the utility value of that node
def maxValue(node):
    #determine the playerChar of the max agent
    terminalTestResult = evaluateTerminalConditionsForStateAndPlayer(node.state, 'X')
    #if the terminal test returned a number, calculate the utility based on the win, lose, or draw
    #outcome of the node
    if terminalTestResult != None:
        utility = 0
        if terminalTestResult > 0: #win
            utility = 10 - node.depth
        elif terminalTestResult < 0: #lose
            utility = node.depth - 10
        node.utility = utility #draw (default)
        return utility
    v = float("-inf")
    nextPossibleStatesForPlayer = generateNextPossibleStatesForStateAndPlayer(node.state,'X')
    #find the max utility value of the results of the next possible moves (min nodes, played by opponent)
    for action in nextPossibleStatesForPlayer:
        v = max(v,minValue(MinMaxNode(node,tuple(action["state"]),action["placement"],None,node.depth+1)))
    node.utility = v
    return v
        
#takes a node and returns the utility value of that node
def minValue(node):
    #do the terminal test based on the max agent
    terminalTestResult = evaluateTerminalConditionsForStateAndPlayer(node.state, 'X')
    #if the terminal test returned a number, calculate the utility based on the win, lose, or draw
    #outcome of the node
    if terminalTestResult != None:
        utility = 0
        if terminalTestResult > 0: #win
            utility = 10 - node.depth
        elif terminalTestResult < 0: #lose
            utility = node.depth - 10
        node.utility = utility #draw (default)
        return utility
    v = float("inf")
    nextPossibleStatesForPlayer = generateNextPossibleStatesForStateAndPlayer(node.state,'O')
    #find the min utility value of the results of the next possible moves (max nodes, played by opponent)
    for action in nextPossibleStatesForPlayer:
        v = min(v,maxValue(MinMaxNode(node,tuple(action["state"]),action["placement"],None,node.depth+1)))
    #update the node's utility 
    node.utility = v
    return v

In [4]:
import random

def start(initialPosition = -1):
    #initialize game
    board = Board()
    #if no parameters are passsed in 
    if initialPosition == -1:
        board.makeMove(Placement(random.choice(range(9)),'X'))
    else:
        board.makeMove(Placement(initialPosition,'X'))
    board.printBoard()
    print()
    currentPlayer = 'O'
    utilityEvaluation  = None
    #take turns until a terminal condition is met (a player wins or the game draws)
    while (not noMoreMovesInState(tuple(board.board))):
        nextMove = minimaxDecision(tuple(board.board),currentPlayer)
        board.makeMove(nextMove)
        board.printBoard()
        print()
        utilityEvaluation = evaluateTerminalConditionsForStateAndPlayer(tuple(board.board), currentPlayer)
        #case that the game has ended in a win, lose, or draw for the current player
        if (utilityEvaluation != None):
            break
        currentPlayer = opposingPlayerTo(currentPlayer) #switch turns
    if utilityEvaluation == 0:
        print("Draw Game")
    elif utilityEvaluation < 0:
        winningPlayer = opposingPlayerTo(currentPlayer)
        print(winningPlayer,"Wins!")
    elif utilityEvaluation > 0:
        print(currentPlayer,"Wins!")
    
    return

You can insert as many cells as you want above
You are not Allowed to modify the code below this line.
# ===============================

In [5]:
#you need to implement print_result function to print out the result according to the required format
start()

***
***
X**

***
*O*
X**

X**
*O*
X**

X**
OO*
X**

X**
OOX
X**

XO*
OOX
X**

XO*
OOX
XX*

XO*
OOX
XXO

XOX
OOX
XXO

Draw Game
